# Demand Profile Metrics Explorer

This notebook loads the demand profile CSV files alongside the dwelling summary data 
to compute the reporting metrics. Adjust the paths below if your repository layout differs.

In [ ]:
from pathlib import Path
import sys

REPO_ROOT = Path('..').resolve()
DEMAND_PROFILES_ROOT = REPO_ROOT / 'Codes' / 'Output' / 'DemandProfiles'
SUMMARY_FILE = REPO_ROOT / 'Codes' / 'Data' / '1R1C1P1S_filtered_filtered.csv'
OUTPUT_FILE = REPO_ROOT / 'Codes' / 'Output' / 'demand_metrics_summary.csv'

if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))

REPO_ROOT, DEMAND_PROFILES_ROOT, SUMMARY_FILE, OUTPUT_FILE

In [ ]:
import pandas as pd

from Codes.generate_demand_metrics import compute_metrics

metrics = compute_metrics(
    demand_profiles_root=DEMAND_PROFILES_ROOT,
    summary_path=SUMMARY_FILE,
)
metrics.head()

In [ ]:
# Save to CSV if desired
metrics.to_csv(OUTPUT_FILE, index=False)
print(f'Saved {len(metrics)} rows to {OUTPUT_FILE}')